### Part A - Movie Similarities

In [1]:
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

In [2]:
def loadMovieNames():
    movieNames = {}
    with open("u.item.txt",encoding="latin-1") as f:
        for line in f:
            fields = line.split('|')
            # print(fields[1])
            movieNames[int(fields[0])] = fields[1].encode().decode()
    return movieNames

def makePairs(user_ratings):
    (movie1, rating1) = user_ratings[1][0]
    (movie2, rating2) = user_ratings[1][1]
    return ((movie1, movie2), (rating1, rating2))

def filterDuplicates( userID_ratingsTuple ):
    # print(userID_ratingsTuple)
    (movie1, rating1) = userID_ratingsTuple[1][0]
    (movie2, rating2) = userID_ratingsTuple[1][1]
    return movie1 < movie2

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [3]:
conf = SparkConf().setMaster("local[*]").setAppName("MovieSimilarities")
sc = SparkContext(conf = conf)

print("\nLoading movie names...")
nameDict = loadMovieNames()

data = sc.textFile("u.data.txt")


Loading movie names...


In [4]:
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

print("\n\n ratings rdd: ")
print(ratings.take(5))
print('\n')



 ratings rdd: 
[(196, (242, 3.0)), (186, (302, 3.0)), (22, (377, 1.0)), (244, (51, 2.0)), (166, (346, 1.0))]




In [6]:
joinedRatings = ratings.join(ratings)
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)

moviePairs = uniqueJoinedRatings.map(makePairs)
moviePairRatings = moviePairs.groupByKey()

moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()
print("\n\n",moviePairSimilarities.take(10),"\n\n")



 [((242, 580), (0.9443699330874624, 6)), ((242, 692), (0.9203762039948743, 18)), ((242, 428), (0.9419097988977888, 15)), ((242, 340), (0.9455404837184603, 32)), ((393, 1241), (1.0, 1)), ((393, 845), (0.9492648155831498, 74)), ((381, 393), (0.9179400502049591, 47)), ((381, 1241), (0.9995120760870788, 2)), ((381, 845), (0.9681290828266734, 30)), ((251, 655), (0.9632240686356973, 17))] 




In [20]:
if (len(sys.argv) > 1):
    scoreThreshold = 0.97
    coOccurenceThreshold = 50
    
#     movieID = 59
    
    for movieID in range(1, 101):
        print("\n", "#####", movieID, "#####")
        
        filteredResults = moviePairSimilarities.filter(lambda pair_sim: \
            (pair_sim[0][0] == movieID or pair_sim[0][1] == movieID) \
            and pair_sim[1][0] > scoreThreshold and pair_sim[1][1] > coOccurenceThreshold)
        print()
        print("\n\n",filteredResults.take(5),"\n\n")

        results = filteredResults.map(lambda pair_sim: (pair_sim[1], pair_sim[0])).sortByKey(ascending = False).take(10)

        print("Top 10 similar movies for " + nameDict[movieID])
        for result in results:
            (sim, pair) = result
            # Display the similarity result that isn't the movie we're looking at
            similarMovieID = pair[0]
            if (similarMovieID == movieID):
                similarMovieID = pair[1]
            print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))
    
    sc.stop()


 ##### 1 #####



 [((1, 169), (0.9718143066672611, 90)), ((1, 969), (0.9734154958854764, 58)), ((1, 174), (0.9740842172192801, 273)), ((1, 318), (0.9704107478975155, 192)), ((1, 210), (0.9714068977811191, 223))] 


Top 10 similar movies for Toy Story (1995)
Hamlet (1996)	score: 0.9745438715121281	strength: 67
Raiders of the Lost Ark (1981)	score: 0.9740842172192801	strength: 273
Cinderella (1950)	score: 0.9740029877471444	strength: 105
Winnie the Pooh and the Blustery Day (1968)	score: 0.9734154958854764	strength: 58
Cool Hand Luke (1967)	score: 0.9733423477201257	strength: 98
Great Escape, The (1963)	score: 0.9732705816130491	strength: 77
African Queen, The (1951)	score: 0.9731512715078089	strength: 101
Apollo 13 (1995)	score: 0.9723951205383821	strength: 207
12 Angry Men (1957)	score: 0.9719872951015222	strength: 81
Wrong Trousers, The (1993)	score: 0.9718143066672611	strength: 90

 ##### 2 #####



 [] 


Top 10 similar movies for GoldenEye (1995)

 ##### 3 #####



 [] 


Top 10 



 [] 


Top 10 similar movies for Rumble in the Bronx (1995)

 ##### 25 #####



 [] 


Top 10 similar movies for Birdcage, The (1996)

 ##### 26 #####



 [((26, 70), (0.9724123902987725, 55))] 


Top 10 similar movies for Brothers McMullen, The (1995)
Four Weddings and a Funeral (1994)	score: 0.9724123902987725	strength: 55

 ##### 27 #####



 [] 


Top 10 similar movies for Bad Boys (1995)

 ##### 28 #####



 [((22, 28), (0.9751504260280339, 181)), ((28, 66), (0.9718399443607805, 99)), ((28, 174), (0.9752200969495095, 242)), ((28, 318), (0.972153895693656, 181)), ((28, 50), (0.9708054799005053, 243))] 


Top 10 similar movies for Apollo 13 (1995)
Hamlet (1996)	score: 0.9773582189340098	strength: 57
Shawshank Redemption, The (1994)	score: 0.9772412307116896	strength: 175
Raiders of the Lost Ark (1981)	score: 0.9752200969495095	strength: 242
Braveheart (1995)	score: 0.9751504260280339	strength: 181
Great Escape, The (1963)	score: 0.974864223139754	strength: 77
Hunt for Red October,



 [] 


Top 10 similar movies for Priest (1994)

 ##### 58 #####



 [((58, 92), (0.9732060770757048, 54)), ((58, 124), (0.9703668096834813, 62)), ((58, 484), (0.9778080588219972, 55)), ((58, 480), (0.9731637048883437, 77)), ((9, 58), (0.9712873538535518, 104))] 


Top 10 similar movies for Quiz Show (1994)
Manchurian Candidate, The (1962)	score: 0.9842058102544757	strength: 59
Wrong Trousers, The (1993)	score: 0.9798872350798274	strength: 52
Maltese Falcon, The (1941)	score: 0.9778080588219972	strength: 55
Butch Cassidy and the Sundance Kid (1969)	score: 0.973460591401908	strength: 88
Casablanca (1942)	score: 0.9733997456758868	strength: 104
Rear Window (1954)	score: 0.973294969543555	strength: 83
True Romance (1993)	score: 0.9732060770757048	strength: 54
North by Northwest (1959)	score: 0.9731637048883437	strength: 77
Platoon (1986)	score: 0.9728125517288128	strength: 65
Four Weddings and a Funeral (1994)	score: 0.972577924214564	strength: 109

 ##### 59 #####



 [((59, 179), (0.97



 [] 


Top 10 similar movies for Robert A. Heinlein's The Puppet Masters (1994)

 ##### 85 #####



 [] 


Top 10 similar movies for Ref, The (1994)

 ##### 86 #####



 [((52, 86), (0.9813832610614274, 57)), ((86, 528), (0.9710697306083039, 55)), ((86, 275), (0.971477179832882, 89)), ((86, 170), (0.9734104054073575, 64)), ((86, 285), (0.970887027453041, 54))] 


Top 10 similar movies for Remains of the Day, The (1993)
Madness of King George, The (1994)	score: 0.9813832610614274	strength: 57
Cinema Paradiso (1988)	score: 0.9734104054073575	strength: 64
Sense and Sensibility (1995)	score: 0.971477179832882	strength: 89
Killing Fields, The (1984)	score: 0.9710697306083039	strength: 55
Secrets & Lies (1996)	score: 0.970887027453041	strength: 54

 ##### 87 #####



 [((87, 523), (0.9794493559722057, 58)), ((87, 603), (0.9721033262838495, 62)), ((87, 479), (0.97123616207553, 58)), ((31, 87), (0.9711921199964219, 62)), ((87, 124), (0.9701509628367333, 54))] 


Top 10 similar movies for Sea

### Part B - Wikipedia

In [7]:
import os, re, json
os.environ['SPARK_HOME'] = 'C:\spark'

import findspark
findspark.init()

from math import log2, log10, ceil
from functools import partial
from pyspark import SparkContext

In [8]:
def ceil5(x):
    return ceil(x/5)*5

def get_winner_loser(match):
    ms = match.split(',')
    return (ms[20], ms[10])

def initialize_voting(losses):
    return {'losses': losses,
            'n_losses': len(losses),
            'rating': 100}

def empty_ratings(d):
    d['rating'] = 0
    return d

def allocate_points(acc, nxt, i):
    k,v = nxt
    
    if i == 0:
        boost = 100 / (v['n_losses']+.01)
    
    else:
        boost = v['rating'] / (v['n_losses'] + .01)
        
    for loss in v['losses']:
        if loss not in acc.keys():
            acc[loss] = {'losses':[], 'n_losses': 0}
        
        opp_rating = acc.get(loss,{}).get('rating',0)
        acc[loss]['rating'] = opp_rating + boost
    
    return acc

def combine_scores(a, b):
    for k,v in b.items():
        try:
            a[k]['rating'] = a[k]['rating'] + b[k]['rating']
        except KeyError:
            a[k] = v
    
    return a

In [9]:
try:
    sc.stop()
except:
    try:
        spark.stop()
    except:
        pass

if __name__ == "__main__":
    sc = SparkContext(appName="WikiMap")
    entries = sc.textFile("wikipedia_edges.txt")
    xs = entries.flatMap(lambda x:x.split('\n'))\
                 .map(lambda x:x.split('\t'))\
                 .groupByKey()\
                 .mapValues(initialize_voting)

    for i in range(7):
        if i > 0:
            xs = sc.parallelize(zs.items())

        acc = dict(xs.mapValues(empty_ratings).collect())
        agg_f = partial(allocate_points, i=i)

        zs = xs.aggregate(acc, agg_f, combine_scores)
        
    ratings = [(k,v['rating']) for k,v in zs.items()]
    
    for player, rating in sorted(ratings, key=lambda x: x[1], reverse=True)[:100]:
        print('{:<30}{}\t{}'.format(player,
                                    round(log2(rating+1), 1),
                                    ceil5(rating)))
    
    sc.stop()

MPQC                          13.3	9940
Penguin (missile)             13.0	8210
List of file systems          13.0	7975
*Lisp                         12.8	7050
Serpent (cipher)              12.4	5475
DirectX Video Acceleration    12.3	4940
John Cocke                    12.2	4635
Bryce (software)              12.1	4315
Fibonacci number              12.0	4090
Deep Blue (chess computer)    12.0	4060
Regular grid                  12.0	4005
Luigi Federico Menabrea       11.9	3955
BiiN                          11.9	3700
Mach (kernel)                 11.8	3595
DirectX                       11.7	3405
Wolfram Mathematica           11.6	3215
Hidden Markov model           11.6	3035
YafaRay                       11.5	2970
Intel                         11.5	2885
Supercomputer                 11.5	2885
Moore's law                   11.5	2865
General-purpose computing on graphics processing units11.5	2865
Lawrence Livermore National Laboratory11.5	2850
Nvidia                        11.5	2815
Mathemat

### Part C - Page Rank

In [1]:
import re
import sys
from operator import add

from pyspark.sql import SparkSession

In [2]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [3]:
try:
    sc.stop()
except:
    try:
        spark.stop()
    except:
        pass

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: pagerank <file> <iterations>", file=sys.stderr)
        sys.exit(-1)

    print("WARN: This is a naive implementation of PageRank and is given as an example!\n" +
          "Please refer to PageRank implementation provided by graphx",
          file=sys.stderr)

    # Initialize the spark context.
    spark = SparkSession.builder.appName("PythonPageRank").getOrCreate()
    
    lines = spark.read.text("pagerank_data.txt").rdd.map(lambda r: r[0])
    links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
    ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))
    
    for iteration in range(10):
        # Calculates URL contributions to the rank of other URLs.
        contribs = links.join(ranks).flatMap(
            lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

        # Re-calculates URL ranks based on neighbor contributions.
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    
    for (link, rank) in ranks.collect():
        print("%s has rank: %s." % (link, rank))
    
    spark.stop()

WARN: This is a naive implementation of PageRank and is given as an example!
Please refer to PageRank implementation provided by graphx


2 has rank: 0.7539975652935547.
3 has rank: 0.7539975652935547.
1 has rank: 1.7380073041193354.
4 has rank: 0.7539975652935547.
